In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import pickle
import PIL.Image as Image
from torchvision.datasets.folder import ImageFolder
import torch.nn.functional as F
from pytorch_msssim import ssim
from skimage import measure
import time

In [2]:
class LeNet(nn.Module):
    def __init__(self, channel=3, hideen=768, num_classes=2):
        super(LeNet, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Conv2d(channel, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=1),
            act(),
        )
        self.fc = nn.Sequential(
            nn.Linear(hideen, num_classes)
        )

    def forward(self, x):
        out = self.body(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def weights_init(m):
    try:
        if hasattr(m, "weight"):
            m.weight.data.uniform_(-0.5, 0.5)
    except Exception:
        print('warning: failed in weights_init for %s.weight' % m._get_name())
    try:
        if hasattr(m, "bias"):
            m.bias.data.uniform_(-0.5, 0.5)
    except Exception:
        print('warning: failed in weights_init for %s.bias' % m._get_name())


class Dataset_from_Image(Dataset):
    def __init__(self, imgs, labs, transform=None):
        self.imgs = imgs # img paths
        self.labs = labs # labs is ndarray
        self.transform = transform
        del imgs, labs

    def __len__(self):
        return self.labs.shape[0]

    def __getitem__(self, idx):
        lab = self.labs[idx]
        img = Image.open(self.imgs[idx])
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = self.transform(img)
        return img, lab

In [3]:

dataset = 'skin'
root_path = '.'
data_path = os.path.join(root_path, './data').replace('\\', '/')
save_path = os.path.join(root_path, 'results/iDLG_%s'%dataset).replace('\\', '/')



''' load data '''
if dataset == 'MNIST':
    shape_img = (28, 28)
    num_classes = 10
    channel = 1
    hidden = 588
    dst = datasets.MNIST(data_path, download=False)


elif dataset == 'skin':
    shape_img = (32, 32)
    num_classes = 2
    channel = 3
    hidden = 768
    dst = ImageFolder(r'C:\Users\badha\OneDrive - Florida International University\Desktop\PhD at FIU\Solid Lab\Spring 2023\CPL Attack Paper Exp\skin11\melanoma_cancer_dataset\train')



else:
    exit('unknown dataset')



lr = 1.0
num_dummy = 1
Iteration = 300
num_exp = 100

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

tt = transforms.Compose([transforms.Resize(32),transforms.CenterCrop(32),transforms.ToTensor()])




tp = transforms.Compose([transforms.ToPILImage()])

print(dataset, 'root_path:', root_path)
print(dataset, 'data_path:', data_path)
print(dataset, 'save_path:', save_path)

if not os.path.exists('results'):
    os.mkdir('results')
if not os.path.exists(save_path):
    os.mkdir(save_path)







skin root_path: .
skin data_path: ././data
skin save_path: ./results/iDLG_skin


In [4]:
# def lfw_dataset(lfw_path, shape_img):
#     images_all = []
#     labels_all = []
#     folders = os.listdir(lfw_path)
#     for foldidx, fold in enumerate(folders):
#         files = os.listdir(os.path.join(lfw_path, fold))
#         for f in files:
#             if len(f) > 4 and f[-4:] == '.jpg':
#                 images_all.append(os.path.join(lfw_path, fold, f))
#                 labels_all.append(foldidx)

#     transform = transforms.Compose([transforms.Resize(size=shape_img)])
#     dst = Dataset_from_Image(images_all, np.asarray(labels_all, dtype=int), transform=transform)
#     return dst



ASR=0
whole_time=0
sSim=0
mSe=0
count=0

''' train DLG and iDLG '''
for idx_net in range(num_exp):
    net = LeNet(channel=channel, hideen=hidden, num_classes=num_classes)
    net.apply(weights_init)

    print('running %d|%d experiment'%(idx_net, num_exp))
    net = net.to(device)
    idx_shuffle = np.random.permutation(len(dst))
    #idx_no_shuffle = np.arange(len(dst))
    start=time.process_time()
    for method in ['iDLG']:#, 'DLG']:
        print('%s, Try to generate %d images' % (method, num_dummy))

        criterion = nn.CrossEntropyLoss().to(device)
        imidx_list = []

        for imidx in range(num_dummy):
            idx = idx_shuffle[imidx]
            imidx_list.append(idx)
            tmp_datum = tt(dst[idx][0]).float().to(device)
            tmp_datum = tmp_datum.view(1, *tmp_datum.size())
            tmp_label = torch.Tensor([dst[idx][1]]).long().to(device)
            tmp_label = tmp_label.view(1, )
            if imidx == 0:
                gt_data = tmp_datum
                gt_label = tmp_label
            else:
                gt_data = torch.cat((gt_data, tmp_datum), dim=0)
                gt_label = torch.cat((gt_label, tmp_label), dim=0)


        # compute original gradient
        out = net(gt_data)
        y = criterion(out, gt_label)
        dy_dx = torch.autograd.grad(y, net.parameters())
        original_dy_dx = list((_.detach().clone() for _ in dy_dx))

        # generate dummy data and label
        dummy_data = torch.randn(gt_data.size()).to(device).requires_grad_(True)
        dummy_label = torch.randn((gt_data.shape[0], num_classes)).to(device).requires_grad_(True)

        if method == 'DLG':
            optimizer = torch.optim.LBFGS([dummy_data, dummy_label])
        elif method == 'iDLG':
            optimizer = torch.optim.LBFGS([dummy_data, ])
            # predict the ground-truth label
            label_pred = torch.argmin(torch.sum(original_dy_dx[-2], dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)

        history = []
        history_iters = []
        losses = []
        mses = []
        mse_f = []
        SSIMS = []
        train_iters = []

        print('lr =', lr)
        for iters in range(Iteration):

            def closure():
                optimizer.zero_grad()
                pred = net(dummy_data)
                if method == 'DLG':
                    dummy_loss = - torch.mean(torch.sum(torch.softmax(dummy_label, -1) * torch.log(torch.softmax(pred, -1)), dim=-1))
                    # dummy_loss = criterion(pred, gt_label)
                elif method == 'iDLG':
                    dummy_loss = criterion(pred, label_pred)

                dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)

                grad_diff = 0
                for gx, gy in zip(dummy_dy_dx, original_dy_dx):
                    grad_diff += ((gx - gy) ** 2).sum()
                grad_diff.backward()
                return grad_diff

            optimizer.step(closure)
            current_loss = closure().item()
            train_iters.append(iters)
            losses.append(current_loss)
            mses.append(torch.mean((dummy_data-gt_data)**2).item())
            mse_f.append(F.mse_loss(dummy_data,torch.unsqueeze(gt_data,dim=0),  reduction='mean').item())
            SSIMS.append(ssim(dummy_data,gt_data,data_range=0).item())


            if iters % int(Iteration / 30) == 0:
                current_time = str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))
                print(current_time, iters, 'loss = %.8f, mse = %.8f, mse_with_function = %.8f, ssim = %.8f' %(current_loss, mses[-1], mse_f[-1], SSIMS[-1]))
                #print(current_time, iters, 'loss = %.8f, mse = %.8f, mse_with_function = %.8f' %(current_loss, mses[-1], mse_f[-1]))
                history.append([tp(dummy_data[imidx].cpu()) for imidx in range(num_dummy)])
                history_iters.append(iters)

                for imidx in range(num_dummy):
                    #plt.figure(figsize=(12, 8))
                    #plt.subplot(3, 10, 1)
                    plt.imshow(tp(gt_data[imidx].cpu()))
                    for i in range(min(len(history), 29)):
                        #plt.subplot(3, 10, i + 2)
                        plt.imshow(history[i][imidx])
                        #plt.title('iter=%d' % (history_iters[i]))
                        plt.axis('off')
                    if method == 'DLG':
                        plt.savefig('%s/DLG_on_%s_%05d.png' % (save_path, imidx_list, imidx_list[imidx]))
                        plt.close()
                    elif method == 'iDLG':
                        plt.savefig('%s/Lbl_%s_iDLG_on_%s_%05d.png' % (save_path, gt_label.item(), imidx_list, imidx_list[imidx]))
                        plt.close()

                if current_loss < 0.000001: # converge
                    break

        if method == 'DLG':
            loss_DLG = losses
            label_DLG = torch.argmax(dummy_label, dim=-1).detach().item()
            mse_DLG = mses
        elif method == 'iDLG':
            loss_iDLG = losses
            label_iDLG = label_pred.item()
            mse_iDLG = mses
            mse_f_iDLG = mse_f
            SSIM = SSIMS

    end=time.process_time()

    print('imidx_list:', imidx_list)
#         print('loss_DLG:', loss_DLG[-1], 'loss_iDLG:', loss_iDLG[-1])
#         print('mse_DLG:', mse_DLG[-1], 'mse_iDLG:', mse_iDLG[-1])
#         print('gt_label:', gt_label.detach().cpu().data.numpy(), 'lab_DLG:', label_DLG, 'lab_iDLG:', label_iDLG)


    print('loss_iDLG:', loss_iDLG[-1])
    print('mse_iDLG:', mse_iDLG[-1])
    print('mse_iDLG_with_functuon:', mse_f_iDLG[-1])
    mSe=mSe+mse_f_iDLG[-1]
    print('SSIM_iDLG:', SSIM[-1])
    sSim=sSim+SSIM[-1]
    duration=end-start
    print('Duration:',duration)
    whole_time=whole_time+duration
    print('gt_label:', gt_label.detach().cpu().data.numpy(), 'lab_iDLG:', label_iDLG)
    
    print('----------------------\n\n')

    if(SSIM[-1]>=.90):
        ASR=ASR+1
        #print(AS)
    count=count+1




running 0|100 experiment
iDLG, Try to generate 1 images
lr = 1.0
[2024-03-21 14:02:50] 0 loss = 202.69508362, mse = 24.24232483, mse_with_function = 24.24232483, ssim = 0.00008449


C:\Users\badha\AppData\Local\Temp\ipykernel_13176\2677447062.py:105: UserWarning: Using a target size (torch.Size([1, 1, 3, 32, 32])) that is different to the input size (torch.Size([1, 3, 32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_f.append(F.mse_loss(dummy_data,torch.unsqueeze(gt_data,dim=0),  reduction='mean').item())


[2024-03-21 14:02:56] 10 loss = 136.31398010, mse = 37.08457184, mse_with_function = 37.08457184, ssim = 0.00012275
[2024-03-21 14:03:04] 20 loss = 119.98109436, mse = 35.76816940, mse_with_function = 35.76816940, ssim = 0.00016621
[2024-03-21 14:03:14] 30 loss = 109.63063812, mse = 34.73434448, mse_with_function = 34.73434448, ssim = 0.00020126
[2024-03-21 14:03:24] 40 loss = 101.53752899, mse = 34.76025009, mse_with_function = 34.76025009, ssim = 0.00019790
[2024-03-21 14:03:35] 50 loss = 94.14621735, mse = 35.73921967, mse_with_function = 35.73921967, ssim = 0.00014325
[2024-03-21 14:03:46] 60 loss = 86.60570526, mse = 36.90452194, mse_with_function = 36.90452194, ssim = 0.00012556
[2024-03-21 14:03:56] 70 loss = 81.68180084, mse = 37.67351532, mse_with_function = 37.67351532, ssim = 0.00015574
[2024-03-21 14:04:06] 80 loss = 77.86107635, mse = 39.38317871, mse_with_function = 39.38317871, ssim = 0.00015497
[2024-03-21 14:04:17] 90 loss = 73.52388000, mse = 41.12364197, mse_with_fun

[2024-03-21 14:10:32] 210 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:32] 220 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:33] 230 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:33] 240 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:33] 250 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:34] 260 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:34] 270 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:34] 280 loss = 124.84951782, mse = 29.16908646, mse_with_function = 29.16908646, ssim = -0.00018867
[2024-03-21 14:10:35] 290 loss = 124.84951782, mse = 29.

[2024-03-21 14:12:59] 120 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:12:59] 130 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:12:59] 140 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:12:59] 150 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:13:00] 160 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:13:00] 170 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:13:00] 180 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:13:01] 190 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.49785650, ssim = -0.00060505
[2024-03-21 14:13:01] 200 loss = 0.00000442, mse = 1.49785650, mse_with_function = 1.497

...

[2024-03-21 16:39:14] 80 loss = 0.00368664, mse = 0.25972542, mse_with_function = 0.25972542, ssim = 0.03569126
[2024-03-21 16:39:24] 90 loss = 0.00244334, mse = 0.20451114, mse_with_function = 0.20451114, ssim = 0.04739698
[2024-03-21 16:39:34] 100 loss = 0.00164931, mse = 0.16149962, mse_with_function = 0.16149962, ssim = 0.06303596
[2024-03-21 16:39:45] 110 loss = 0.00107178, mse = 0.12489109, mse_with_function = 0.12489109, ssim = 0.08164959
[2024-03-21 16:39:55] 120 loss = 0.00069898, mse = 0.09603649, mse_with_function = 0.09603649, ssim = 0.10583860
[2024-03-21 16:40:05] 130 loss = 0.00046044, mse = 0.07468708, mse_with_function = 0.07468708, ssim = 0.13390574
[2024-03-21 16:40:15] 140 loss = 0.00031595, mse = 0.05880284, mse_with_function = 0.05880284, ssim = 0.16550425
[2024-03-21 16:40:26] 150 loss = 0.00023170, mse = 0.04759269, mse_with_function = 0.04759269, ssim = 0.19682160
[2024-03-21 16:40:37] 160 loss = 0.00016930, mse = 0.03807054, mse_with_function = 0.03807054, ssi

[2024-03-21 16:43:15] 150 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:15] 160 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:16] 170 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:16] 180 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:16] 190 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:17] 200 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:17] 210 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002
[2024-03-21 16:43:17] 220 loss = 0.00354154, mse = 3554309.50000000, mse_with_function = 3554309.50000000, ssim = 0.00000002


[2024-03-21 16:46:21] 170 loss = 0.00000312, mse = 0.00022003, mse_with_function = 0.00022003, ssim = 0.65821600
[2024-03-21 16:46:32] 180 loss = 0.00000216, mse = 0.00015437, mse_with_function = 0.00015437, ssim = 0.71231860
[2024-03-21 16:46:42] 190 loss = 0.00000145, mse = 0.00010211, mse_with_function = 0.00010211, ssim = 0.76864445
[2024-03-21 16:46:53] 200 loss = 0.00000102, mse = 0.00007377, mse_with_function = 0.00007377, ssim = 0.80674446
[2024-03-21 16:46:57] 210 loss = 0.00000093, mse = 0.00006873, mse_with_function = 0.00006873, ssim = 0.81429708
imidx_list: [4303]
loss_iDLG: 9.260704700864153e-07
mse_iDLG: 6.873169331811368e-05
mse_iDLG_with_functuon: 6.873169331811368e-05
SSIM_iDLG: 0.814297080039978
Duration: 204.03125
gt_label: [0] lab_iDLG: 0
----------------------


running 95|100 experiment
iDLG, Try to generate 1 images
lr = 1.0
[2024-03-21 16:46:57] 0 loss = 0.00000445, mse = 1.60659480, mse_with_function = 1.60659480, ssim = 0.00023452
[2024-03-21 16:46:57] 10 los

[2024-03-21 16:47:23] 10 loss = 0.66629952, mse = 0.59472823, mse_with_function = 0.59472823, ssim = 0.01148579
[2024-03-21 16:47:33] 20 loss = 0.09204745, mse = 0.31898171, mse_with_function = 0.31898171, ssim = 0.02445525
[2024-03-21 16:47:44] 30 loss = 0.02822004, mse = 0.20398013, mse_with_function = 0.20398013, ssim = 0.03674884
[2024-03-21 16:47:54] 40 loss = 0.01134707, mse = 0.13641731, mse_with_function = 0.13641731, ssim = 0.05268940
[2024-03-21 16:48:04] 50 loss = 0.00543268, mse = 0.09385577, mse_with_function = 0.09385577, ssim = 0.07951735
[2024-03-21 16:48:15] 60 loss = 0.00278460, mse = 0.06427372, mse_with_function = 0.06427372, ssim = 0.10737520
[2024-03-21 16:48:25] 70 loss = 0.00156186, mse = 0.04578448, mse_with_function = 0.04578448, ssim = 0.12760358
[2024-03-21 16:48:36] 80 loss = 0.00089945, mse = 0.03213467, mse_with_function = 0.03213467, ssim = 0.14738300
[2024-03-21 16:48:46] 90 loss = 0.00052655, mse = 0.02230355, mse_with_function = 0.02230355, ssim = 0.1

[2024-03-21 16:52:52] 60 loss = 0.00341230, mse = 0.11567696, mse_with_function = 0.11567696, ssim = 0.05475277
[2024-03-21 16:53:03] 70 loss = 0.00130614, mse = 0.06825338, mse_with_function = 0.06825338, ssim = 0.09297214
[2024-03-21 16:53:13] 80 loss = 0.00062837, mse = 0.04461291, mse_with_function = 0.04461291, ssim = 0.13841084
[2024-03-21 16:53:24] 90 loss = 0.00035199, mse = 0.02932766, mse_with_function = 0.02932766, ssim = 0.20454769
[2024-03-21 16:53:34] 100 loss = 0.00018192, mse = 0.02048240, mse_with_function = 0.02048240, ssim = 0.26694477
[2024-03-21 16:53:45] 110 loss = 0.00011253, mse = 0.01417370, mse_with_function = 0.01417370, ssim = 0.33628270
[2024-03-21 16:53:55] 120 loss = 0.00007476, mse = 0.01001811, mse_with_function = 0.01001811, ssim = 0.40373415
[2024-03-21 16:54:05] 130 loss = 0.00005048, mse = 0.00690067, mse_with_function = 0.00690067, ssim = 0.47633290
[2024-03-21 16:54:16] 140 loss = 0.00003166, mse = 0.00463656, mse_with_function = 0.00463656, ssim 

In [5]:
print("Count:",count)
print("ASR:",ASR/count)
print("Avg. SSIM:",sSim/count)
print("Avg. MSE is:",mSe/count)
print("Avg. time:",whole_time/count)

Count: 100
ASR: 0.36
Avg. SSIM: 0.4280079014971907
Avg. MSE is: 4.0112394911629843e+24
Avg. time: 102.60015625
